# Fase 5 — EDA, Clusterização e Regressão
**Aluno:** Tiago Lopes (rmXXXXX) — **Projeto:** FarmTech Fase 5

> Coloque o arquivo `../data/crop_yield.csv` antes de executar.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

try:
    import xgboost as xgb
    HAS_XGB = True
except Exception:
    HAS_XGB = False

# Configs
pd.set_option('display.max_columns', None)
sns.set()

## 1) Carregamento do Dataset

In [ ]:
# Ajuste o caminho se necessário
DATA_PATH = '../data/crop_yield.csv'
df = pd.read_csv(DATA_PATH)

print('Shape:', df.shape)
display(df.head())

## 2) EDA — Estrutura, Resumo e Nulos

In [ ]:
display(df.info())
display(df.describe().T)

nulos = df.isna().sum().sort_values(ascending=False)
display(nulos.to_frame('missing'))

### 2.1) Histogramas das Variáveis Numéricas

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for col in num_cols:
    plt.figure()
    df[col].hist(bins=30)
    plt.title(f'Histograma: {col}')
    plt.xlabel(col)
    plt.ylabel('Frequência')
    plt.show()

### 2.2) Boxplots para Outliers

In [ ]:
for col in num_cols:
    plt.figure()
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot: {col}')
    plt.show()

### 2.3) Scatterplots (Ex.: temperatura × rendimento)

In [ ]:
# Tente adivinhar colunas comuns; ajuste para o seu dataset
candidate_x = None
for name in ['temperatura','temperature','temp','avg_temp','t_avg']:
    if name in df.columns:
        candidate_x = name
        break

candidate_y = None
for name in ['rendimento','yield','yld','produtividade']:
    if name in df.columns:
        candidate_y = name
        break

if candidate_x and candidate_y:
    plt.figure()
    plt.scatter(df[candidate_x], df[candidate_y], alpha=0.6)
    plt.xlabel(candidate_x)
    plt.ylabel(candidate_y)
    plt.title(f'{candidate_x} × {candidate_y}')
    plt.show()
else:
    print('⚠️ Ajuste os nomes das colunas para fazer o scatterplot.')

## 3) Clusterização — KMeans (k=3..5)

In [ ]:
# Seleciona apenas colunas numéricas para clustering
X_num = df.select_dtypes(include=[np.number]).dropna()

# Normalização (opcional, geralmente ajuda)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_num)

# Escolha de 3..5 clusters; você pode ajustar
best_inertia = None
best_k = None
best_model = None

for k in [3,4,5]:
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(X_scaled)
    if best_inertia is None or kmeans.inertia_ < best_inertia:
        best_inertia = kmeans.inertia_
        best_k = k
        best_model = kmeans

labels = best_model.labels_
X_num_clustered = X_num.copy()
X_num_clustered['cluster'] = labels
display(X_num_clustered.head())

print(f'Melhor k escolhido pelo menor inertia: {best_k} (inertia={best_inertia:.2f})')

In [ ]:
# Scatter simples usando as duas primeiras features numéricas
if X_num.shape[1] >= 2:
    cols = X_num.columns[:2]
    plt.figure()
    sns.scatterplot(x=X_num[cols[0]], y=X_num[cols[1]], hue=labels, palette='tab10')
    plt.title(f'Dispersão por clusters (k={best_k})')
    plt.show()
else:
    print('⚠️ São necessárias pelo menos 2 colunas numéricas para o gráfico de dispersão.')

> **Outliers potencialmente isolados:** verifique se algum cluster apresenta valores extremos de rendimento/produção.  
Dica: analise as estatísticas por cluster:

In [ ]:
# Estatísticas por cluster para inspecionar outliers
cluster_stats = X_num_clustered.groupby('cluster').agg(['mean','std','min','max'])
display(cluster_stats)

## 4) Modelagem Preditiva — Regressão Supervisionada

In [ ]:
# Defina a variável alvo (y)
# Tente mapear nomes comuns; ajuste para o seu dataset:
y_col = None
for name in ['rendimento','yield','yld','produtividade']:
    if name in df.columns:
        y_col = name
        break

if y_col is None:
    raise ValueError('Defina manualmente a coluna alvo (ex.: y_col = "yield")')

X = df.drop(columns=[y_col])
# One-hot para categóricas
X = pd.get_dummies(X, drop_first=True)

# Remove linhas com NA após dummies
mask = X.notna().all(axis=1) & df[y_col].notna()
X = X[mask]
y = df.loc[mask, y_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'LinearRegression': LinearRegression(),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=300, random_state=42),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.001),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
}

if HAS_XGB:
    models['XGBoost'] = xgb.XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

def eval_model(name, model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    r2 = r2_score(y_test, pred)
    rmse = mean_squared_error(y_test, pred, squared=False)
    mae = mean_absolute_error(y_test, pred)
    return {'model': name, 'R2': r2, 'RMSE': rmse, 'MAE': mae}

results = []
for name, model in models.items():
    try:
        results.append(eval_model(name, model, X_train, X_test, y_train, y_test))
    except Exception as e:
        results.append({'model': name, 'R2': np.nan, 'RMSE': np.nan, 'MAE': np.nan, 'error': str(e)})

results_df = pd.DataFrame(results).sort_values(by=['R2','RMSE'], ascending=[False, True])
display(results_df)

## 5) Conclusões

- **EDA:** descreva aqui os principais achados (tendências, correlações, nulos, outliers).
- **Clusterização:** destaque se algum cluster representa potenciais outliers em rendimento.
- **Modelagem:** indique qual modelo teve melhor desempenho e por quê (viés/variância, interpretabilidade, etc.).
- **Limitações:** tamanho do dataset, variáveis limitadas, qualidade dos dados, ausência de tuning, etc.